In [1]:
### 计算相似度
from collections import Counter
from typing import List
def get_alignment(de_raw_sentence: List[str], embedding_type):
    de_alignment = []
    id = 0
    de_raw_tokens = de_raw_sentence.split()
    for word in de_raw_tokens:
        de_alignment.extend([id for _ in range(len(get_tokenized_tokens(word, embedding_type)))])
        id += 1
#     print(de_alignment, len(de_alignment))
    assert id == len(de_raw_tokens), (id, len(de_raw_tokens))
    de_alignment_dict = Counter(de_alignment)
    return de_alignment, de_alignment_dict


def cosine_distance(a, b):
    if a.shape != b.shape:
        raise RuntimeError("array {} shape not match {}".format(a.shape, b.shape))
    if a.ndim==1:
        a_norm = np.linalg.norm(a)
        b_norm = np.linalg.norm(b)
    elif a.ndim==2:
        a_norm = np.linalg.norm(a, axis=1, keepdims=True)
        b_norm = np.linalg.norm(b, axis=1, keepdims=True)
    else:
        raise RuntimeError("array dimensions {} not right".format(a.ndim))
    similiarity = np.dot(a, b.T)/(a_norm * b_norm)
    dist = 1. - similiarity
    return dist



def get_aligned_embeddings(de_alignment, de_alignment_dict, de_sentence_embed):
    id, de_emb_average, alignment_de_embeds = 0, 0., []
    
    for i, de_emb in enumerate(de_sentence_embed):
        ### apply alignment
        if de_alignment[i] == id:
            de_emb_average += de_emb
        else:
            de_emb_average /= de_alignment_dict[id]
            alignment_de_embeds.append(de_emb_average)
            id += 1
            de_emb_average = de_emb
    de_emb_average  /= de_alignment_dict[id]
    alignment_de_embeds.append(de_emb_average)
    assert len(alignment_de_embeds) == max(de_alignment)+1, (len(alignment_de_embeds), max(de_alignment)+1)
    return alignment_de_embeds
                        
def compute_word2words_similarity(keys, query, sim_type=1,
                                  norm_similarity=False, k=2, sparse_top_k=False):
    similarity = []
    for i, key in enumerate(keys):
        if sim_type == 1: similarity.append(1-cosine_distance(query.cpu(), key.cpu()))
        elif sim_type == 2: similarity.append(np.exp(-np.linalg.norm(query-key, ord=2, axis=-1)))
        else: e
    
    if sparse_top_k:
        top_k_similarity = sorted(similarity, reverse=True)[k-1]
        similarity = [i if i >= top_k_similarity else 0 for i in similarity]
    if norm_similarity:
        similarity = np.array(similarity) / np.array(similarity).sum()
        similarity = similarity.tolist()
    
    return similarity

In [2]:
#获取分词工具
from bert_tokenizer import BertTokenizer
tokenizer = BertTokenizer.load_from_vocab_file('/home/sxy/Projects/CP/OVC-MMT/data/MY_MULT30K/distil_cased_based/vocab.txt')
tokenizer._cased = True
multi_bert_tokenizer = BertTokenizer.load_from_vocab_file('/home/sxy/Projects/CP/OVC-MMT/data/MY_MULT30K/multi_cased_L-12_H-768_A-12/vocab.txt')
multi_bert_tokenizer._cased = True

In [3]:
## mbert
from sentence_transformers import SentenceTransformer
embedding_type = 4 # 1 for static word embedding and 2 for dynamic token embedding

### bert model
## distiluse bert 
embedder = SentenceTransformer('distiluse-base-multilingual-cased')#'bert-base-nli-mean-tokens')

In [4]:
## 词向量工具
import numpy as np

def get_tokenized_tokens(string, embedding_type):
    if embedding_type == 1: en_temp = [string]
    elif embedding_type == 2: en_temp = tokenizer.tokenize(string)
    else: en_temp = multi_bert_tokenizer.tokenize(string)
    return en_temp



def get_embedding(string : str, embedding_type=1):
    if embedding_type == 1:   # seq_len * embed_dim
        return [model[w] for w in tokenizer.tokenize(string)]
    elif embedding_type == 2: # seq_len * embed_dim
        return [(embedder.encode(w, output_value = 'token_embeddings')[0]).mean(axis=0) for w in string.split()][:len(string.split())]
        return [(embedder.encode(w, output_value = 'token_embeddings')[0][1:-1]).mean(axis=0) for w in string.split()]
    elif embedding_type == 3: # seq_len * embed_dim
        return embedder.encode(string, output_value = 'token_embeddings')[1:-1]
    elif embedding_type == 4: # seq_len * embed_dim
        return embedder.encode(string, output_value = 'token_embeddings').mean(axis=0)
    elif embedding_type == 5: # seq_len * embed_dim
        embs = []
        for w in string.split():
            emb, sub_ws = 0., multi_bert_tokenizer.tokenize(w)
            print(sub_ws)
#             emb = (embedder.encode(sub_ws[0], output_value = 'token_embeddings')).mean(axis=0)
            for sub_w in sub_ws: emb += (embedder.encode(sub_w, output_value = 'token_embeddings')).mean(axis=0)
            embs.append(emb/float(len(sub_ws)))
#             embs.append(emb)
        return embs
    elif embedding_type == 6:
        embs = []
        for w in string.split():
            emb = (embedder.encode(w, output_value = 'token_embeddings')).mean(axis=0)
            embs.append(emb)
        return embs
    elif embedding_type == 7:
        embs = []
        for w in string.split():
            emb, sub_ws = 0., multi_bert_tokenizer.tokenize(w)
            print(sub_ws)
            for sub_w in sub_ws:
                try:
                    emb += (model[sub_w])
                except:
                    emb += np.full(768,0)
                    continue
            embs.append(emb/float(len(sub_ws)))
        return embs

In [67]:
## 获取obj信息
import numpy as np
spilit = 'train'
npzfile = '/home/sxy/Projects/CP/OVC-MMT/img_out/flickr30k-images/'
items_obj = []
items_att = []
with open('/home/sxy/Projects/CP/OVC-MMT/data/MY_MULT30K/%s.txt' % spilit, 'r') as f:
    for i in f.readlines():
        file = i.split('.')[0]
        it = np.load(npzfile+file+'.npz',allow_pickle=True)
        temp_obj = it['info'].tolist()['objects_id'].tolist()
        temp_att = it['info'].tolist()['attrs_id'].tolist()
        items_obj.append(temp_obj)
        items_att.append(temp_att)
attributes = []
classes = [] 
with open('/home/sxy/Projects/CP/OVC-MMT/scripts/raw_data/vcr/attributes_vocab.txt', 'r') as f1:
    for att in f1.readlines():
        attributes.append(att.split(',')[0].lower().strip())
    
with open('/home/sxy/Projects/CP/OVC-MMT/scripts/raw_data/vcr/objects_vocab.txt', 'r') as f2: 
    for object in f2.readlines():
        classes.append(object.split(',')[0].lower().strip())



def get_object_list(temp_obj_list,temp_att_list):
    all_list = []
    
    for i,j in zip(temp_att_list,temp_obj_list):
        all_list.append(attributes[i]+' '+classes[j])
    return all_list


obj_all_list = []#存放每一句话的所有信息
for obj,att in zip(items_obj,items_att):
        line = get_object_list(obj,att)
        obj_all_list.append(line)

In [68]:
# sentence_path = '/home/sxy/Projects/CP/multi30k-wmt18/data/task1/tok/test_2017_flickr.lc.norm.tok.en'
sentence_path = '/home/sxy/Projects/CP/multi30k-wmt18/data/task1/tok/train.lc.norm.tok.en'
sentence_list = []
with open(sentence_path, 'r',) as r:
    for line in r.readlines():
        sentence_list.append(line.split())
print(len(max(sentence_list,key = len)))

40


In [69]:
import numpy as np
matrix = np.lib.format.open_memmap(
        '/home/sxy/Projects/CP/work/data-bin/muti30k_en_de_bpe10k/' + "train_matrix0.8.npy", mode="w+",
        dtype=np.float16, shape=(len(sentence_list),50,10))
import torch
test = torch.zeros(50,10)#初始化一个50*10的矩阵

In [ ]:
import copy
threshold = 0.8
embedding_type, align_similairty = 3, False
if embedding_type == 3: align_similairty = True
index = 0
for sentence,obj in zip(sentence_list,obj_all_list):#遍历每一行
#     print(sentence)
    for sentence_index in range(len(sentence)):#遍历每一个词
#         print(sentence[sentence_index])
        now_sentence_embd = get_embedding(sentence[sentence_index], embedding_type)#获取当前词bert向量
        now_sentence_token = tokenizer.tokenize(sentence[sentence_index])#获取当前词分词
        for obj_index in range(len(obj)):
            now_obj_embd = get_embedding(obj[obj_index], embedding_type)#获取当前词bert向量,一个特征两个词
            obj_alignment, obj_alignment_dict = get_alignment(obj[obj_index], embedding_type)#分词列表
#             print(obj[obj_index])
            keys = now_obj_embd
            map_dict = {}
            for txt, query in zip(now_sentence_token, now_sentence_embd):
                similarity = compute_word2words_similarity(keys, query, sim_type=1)#获取一个相似度列表
                if align_similairty:
                    new_similarity = []
                    for id in range(max(obj_alignment)+1):
                        similarity_temp = [similarity[i] for i, a in enumerate(obj_alignment) if a == id]
    #                     print(similarity_temp)
                        if len(similarity_temp) == 0:
                            new_similarity.append([])
                        else:
                            new_similarity.append(max(similarity_temp))
                    similarity = new_similarity
                old_imilarity = 0
#                 print('--------------------')
#                 print(obj[obj_index])
                for a, b in zip(obj[obj_index].split(), similarity):
#                     print(sentence[sentence_index])
                    if b >= threshold and b > old_imilarity:#如果相似度对应的单词阈值大于0.7，且比其他的大，更新对应词表
                        matrix[index][sentence_index][obj_index] = 1
                        break
    index = index + 1
matrix.flush()
                        
            
    

In [43]:
## 获取obj信息
import numpy as np
spilit = 'val'
npzfile = '/home/sxy/Projects/CP/OVC-MMT/img_out/flickr30k-images/'
items_obj = []
items_att = []
with open('/home/sxy/Projects/CP/OVC-MMT/data/MY_MULT30K/%s.txt' % spilit, 'r') as f:
    for i in f.readlines():
        file = i.split('.')[0]
        it = np.load(npzfile+file+'.npz',allow_pickle=True)
        temp_obj = it['info'].tolist()['objects_id'].tolist()
        temp_att = it['info'].tolist()['attrs_id'].tolist()
        items_obj.append(temp_obj)
        items_att.append(temp_att)
attributes = []
classes = [] 
with open('/home/sxy/Projects/CP/OVC-MMT/scripts/raw_data/vcr/attributes_vocab.txt', 'r') as f1:
    for att in f1.readlines():
        attributes.append(att.split(',')[0].lower().strip())
    
with open('/home/sxy/Projects/CP/OVC-MMT/scripts/raw_data/vcr/objects_vocab.txt', 'r') as f2: 
    for object in f2.readlines():
        classes.append(object.split(',')[0].lower().strip())



def get_object_list(temp_obj_list,temp_att_list):
    all_list = []
    
    for i,j in zip(temp_att_list,temp_obj_list):
        all_list.append(attributes[i]+' '+classes[j])
    return all_list


obj_all_list = []#存放每一句话的所有信息
for obj,att in zip(items_obj,items_att):
        line = get_object_list(obj,att)
        obj_all_list.append(line)

In [44]:
# sentence_path = '/home/sxy/Projects/CP/multi30k-wmt18/data/task1/tok/test_2017_flickr.lc.norm.tok.en'
sentence_path = '/home/sxy/Projects/CP/multi30k-wmt18/data/task1/tok/val.lc.norm.tok.fr'
sentence_list = []
with open(sentence_path, 'r',) as r:
    for line in r.readlines():
        sentence_list.append(line.split())
print(len(max(sentence_list,key = len)))

36


In [45]:
fr_o_list = []

In [46]:
import copy
import json
threshold = 0.4
embedding_type, align_similairty = 3, False
if embedding_type == 3: align_similairty = True
index = 0
for sentence,obj in zip(sentence_list,obj_all_list):#遍历每一行
    sentence_list_dict = {}
#     print(sentence)
    for sentence_index in range(len(sentence)):#遍历每一个词
#         print(sentence[sentence_index])
        now_sentence_embd = get_embedding(sentence[sentence_index], embedding_type)#获取当前词bert向量
        now_sentence_token = tokenizer.tokenize(sentence[sentence_index])#获取当前词分词
        for obj_index in range(len(obj)):
            now_obj_embd = get_embedding(obj[obj_index], embedding_type)#获取当前词bert向量,一个特征两个词
            obj_alignment, obj_alignment_dict = get_alignment(obj[obj_index], embedding_type)#分词列表
#             print(obj[obj_index])
            keys = now_obj_embd
            map_dict = {}
            for txt, query in zip(now_sentence_token, now_sentence_embd):
                similarity = compute_word2words_similarity(keys, query, sim_type=1)#获取一个相似度列表
                if align_similairty:
                    new_similarity = []
                    for id in range(max(obj_alignment)+1):
                        similarity_temp = [similarity[i] for i, a in enumerate(obj_alignment) if a == id]
    #                     print(similarity_temp)
                        if len(similarity_temp) == 0:
                            new_similarity.append([])
                        else:
                            new_similarity.append(max(similarity_temp))
                    similarity = new_similarity
                old_imilarity = 0
#                 print('--------------------')
#                 print(obj[obj_index])
                for a, b in zip(obj[obj_index].split(), similarity):
#                     print(sentence[sentence_index])
                    if b >= threshold and b > old_imilarity:#如果相似度对应的单词阈值大于0.7，且比其他的大，更新对应词表
                        if sentence_index in sentence_list_dict:
                            sentence_list_dict[sentence_index].append(obj_index)
                        else:
                            sentence_list_dict[sentence_index] = [copy.deepcopy(obj_index)]
    fr_o_list.append(sentence_list_dict)
out = '/home/sxy/Projects/CP/OVC-MMT/data/MY_MULT30K/fr/valid_fr_o0.4.txt'
with open(out,'w') as w:
    for i in fr_o_list:
        item = json.dumps(i,ensure_ascii=False)
        w.write(item)
        w.write('\n')

In [111]:
a = {}
a[1] = [1,2,3,4,5,6]
a[1].append(7)
a

{1: [1, 2, 3, 4, 5, 6, 7]}